In [2]:
import ee


# Initialize the Earth Engine module.
ee.Initialize(project="ee-sakda-451407")

In [ ]:
import ee
import folium

# Initialize the Earth Engine API
ee.Initialize(project="ee-sakda-451407")


# Define region
region = ee.Geometry.Point([99.0, 18.8])

# Load full CHIRPS dataset (2010–2023)
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate('2010-01-01', '2023-12-31') \
    .filterBounds(region) \
    .select('precipitation')

# Function to create monthly sum images
def create_monthly_images(year):
    year = ee.Number(year)
    months = ee.List.sequence(1, 12)

    def create_monthly_image(month):
        month = ee.Number(month)
        start = ee.Date.fromYMD(year, month, 1)
        end = start.advance(1, 'month')
        monthly_image = chirps.filterDate(start, end).sum() \
            .set('system:time_start', start.millis())
        return monthly_image

    return months.map(create_monthly_image)

# Generate monthly images for 2010–2023
years = ee.List.sequence(2010, 2023)
monthly = ee.ImageCollection(years.map(create_monthly_images).flatten())

# Convert monthly images to features with date + mean rainfall
def image_to_feature(img):
    img = ee.Image(img)
    mean_dict = img.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=region,
        scale=5000
    )
    return ee.Feature(None, {
        'rainfall': mean_dict.get('precipitation'),
        'date': img.date().format('YYYY-MM-dd')
    })

timeseries = monthly.map(image_to_feature)

# Export the FeatureCollection to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection(timeseries),
    description='rainfall_timeseries',
    fileFormat='CSV',
    folder='_GEE',
    fileNamePrefix='rainfall_timeseries',
)

task.start()
print("Export started. Check your Earth Engine Tasks tab or use task.status() to monitor.")

# Get the first image and visualize it
first_image = ee.Image(monthly.first())

visualize = {
    'min': 0, 
    'max': 100,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Add Earth Engine layer support to folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Monkey patch the method
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map centered on Chiang Mai
m = folium.Map(location=[18.8, 99.0], zoom_start=8)
m.add_ee_layer(first_image, visualize, 'Monthly Precipitation')
m.add_child(folium.LayerControl())

m


In [15]:
point = ee.Geometry.Point([-122.262, 37.8719])
buffer = point.buffer(1000)
print(buffer.getInfo())

m = folium.Map(location=[37.8719, -122.262], zoom_start=12)
folium.GeoJson(
        buffer.getInfo(),
        style_function=lambda x: {'color': 'blue', 'weight': 2, 'fillOpacity': 0.1}
    ).add_to(m)
m


{'type': 'Polygon', 'coordinates': [[[-122.262, 37.88089886595148], [-122.26518450137381, 37.88054066820286], [-122.26811542959013, 37.87949459747022], [-122.27055942303024, 37.87784394778709], [-122.27232192584361, 37.87572014613351], [-122.27326267894644, 37.87329227983932], [-122.27330687335235, 37.87075362702179], [-122.2724510815639, 37.86830626542153], [-122.27076350219738, 37.86614498671441], [-122.26837850717031, 37.86444179681054], [-122.26548593304163, 37.86333223413037], [-122.26231597285279, 37.86290459181972], [-122.25912087019036, 37.86319289836205], [-122.25615486718524, 37.86417421260515], [-122.25365399430027, 37.86577044736679], [-122.25181730207703, 37.8678545772425], [-122.25079102263011, 37.8702607389464], [-122.25065691978158, 37.87279742359074], [-122.25142575821114, 37.875262714055125], [-122.25303641847457, 37.87746035665553], [-122.25536073710724, 37.87921538810274], [-122.2582136943497, 37.880388072234915], [-122.26136814295806, 37.88088503414734], [-122.262,

In [13]:
!pip install folium